# Leetcode Problems in Kotlin

## 11. Container With Most Water

https://leetcode.com/problems/container-with-most-water/

Given `n` non-negative integers `a1, a2, ..., an` , where each represents a point at coordinate `(i, ai)`. `n` vertical lines are drawn such that the two endpoints of the line `i` is at `(i, ai)` and `(i, 0)`. Find two lines, which, together with the x-axis forms a container, such that the container contains the most water.

Notice that you may not slant the container.

### Main Idea: Two Pointer
We want to have one pointer start at the beginning, the other start at the end of the array, and keep track of the maximum area we've seen. In each iteration, compute the area and check to see if it is greater than what we've already found. Then, we want to move the index with the shorter height further into the array.

$O(n)$ running time, $O(1)$ space

In [1]:
class Solution11() {
    fun maxArea(height: Array<Int>) : Int {
        var currentMax = 0
        var left = 0
        var right = height.size - 1
        
        while (left < right) {
            val width = right - left
            val minHeight = if (height[left] <= height[right]) height[left] else height[right]
            val area = width * minHeight
            
            if (area > currentMax) {
                currentMax = area;
            }
            
            if (minHeight == height[left]) {
                left++
            } else {
                right--
            }
        }
        
        return currentMax
    }
    
    fun test() {
        var height = arrayOf(1,8,6,2,5,4,8,3,7)
        checkResult(height, maxArea(height), 49)
        
        height = arrayOf(1,1)
        checkResult(height, maxArea(height), 1)
        
        height = arrayOf(4,3,2,1,4)
        checkResult(height, maxArea(height), 16)
        
        height = arrayOf(1,2,1)
        checkResult(height, maxArea(height), 2)
        
        height = arrayOf(0,2)
        checkResult(height, maxArea(height), 0)
        
        height = arrayOf(2,3,4,5,18,17,6)
        checkResult(height, maxArea(height), 17)        
    }
    
    fun checkResult(height: Array<Int>, result: Int, expected: Int) {
        println("Input: ${height.joinToString(",")}")
        println("Expected: $expected | Actual: $result")
    }
}

val sol11 = Solution11()
sol11.test()

Input: 1,8,6,2,5,4,8,3,7
Expected: 49 | Actual: 49
Input: 1,1
Expected: 1 | Actual: 1
Input: 4,3,2,1,4
Expected: 16 | Actual: 16
Input: 1,2,1
Expected: 2 | Actual: 2
Input: 0,2
Expected: 0 | Actual: 0
Input: 2,3,4,5,18,17,6
Expected: 17 | Actual: 17


## 36. Valid Sudoku

https://leetcode.com/problems/valid-sudoku/

Determine if a `9 x 9` Sudoku board is valid. Only the filled cells need to be validated **according to the following rules**:

- Each row must contain the digits `1-9` without repetition.
- Each column must contain the digits `1-9` without repetition.
- Each of the nine `3 x 3` sub-boxes of the grid must contain the digits `1-9` without repetition.

**Note**:

- A Sudoku board (partially filled) could be valid but is not necessarily solvable.
- Only the filled cells need to be validated according to the mentioned rules.

### Main Idea

We want to traverse the matrix and continously check if the row/column/subbox are all valid. I create 3 arrays of 9 Sets. When we encounter a sudoku character, we try adding it into the appropriate Set, return false if that fails.

Being able to retrieve the Sets for rows and columns is straightforward enough, we just use the appropriate index. To retrieve the Set for the subbox, I use the following formula: 
$$ index = \frac{col}{3} + 3 * \frac{row}{3} $$

**I have a feeling I'm good to need to be able to derive such formulas whenever I'm working with matrices so it might be worth practicing a few more of these.**

Another piece of the puzzle is regarding how we want to traverse the matrix. The way I did it is more of a speed optimization and I think that it adds a fair amount of complexity in order to make the solution faster. That said, the reason I went straight for this is because I thought Leetcode would give me a Time Limit Exceeded error if all I did was `for row in 0 to 9 ... for column in 0 to 9`. 

Anyway, I decided to traverse the matrix by keeping track of the `diagonalIndex` in the outer loop, and a counter variable `i` in the inner loop that starts from whatever the current value of the `diagonalIndex` is. In one `diagonalIndex` iteration, I check the validity of the entire row and the entire column which has the point `[diagonalIndex, diagonalIndex]`; indeed, this is our starting point in the inner loop. As a result, we shrink the board in both the `x` and `y` dimensions after each outer loop iteration. In the process of checking each row/column in one outer loop iteration, we encounter multiple different subboxes; we check the validity of these subboxes too.

This is the general outline of the solution, let's look at the implementation.

### Implementation

In [4]:
class Solution36() {
    fun isValidSudoku(board: Array<Array<Char>>) : Boolean {
        var isValid = true
        var diagonalIndex = 0
        var row = 0
        var column = 0
        
        var boxSets = Array<HashSet<Char>>(9)
        var rowSets = Array<HashSet<Char>>(9)
        var colSets = Array<HashSet<Char>>(9)
        for (i in 0 until 9) {
            boxSets[i] = HashSet<Char>()
            rowSets[i] = HashSet<Char>()
            colSets[i] = HashSet<Char>()
        }
        
        while (diagonalIndex < 9 && isValid) {
            row = diagonalIndex
            column = diagonalIndex
            val rowSet = rowSets[row]
            val colSet = colSets[column]
            
            // check the row/col
            for (i in diagonalIndex until 9) {
                val rowCurrent = board[row][i]
                val colCurrent = board[i][column]
                
                val isRowDot = rowCurrent == '.'
                val isColDot = colCurrent == '.'
                
                if (!isRowDot && !rowSet.add(rowCurrent)) {
                    isValid = false
                    break
                }
                
                if (!isColDot && !colSet.add(colCurrent)) {
                    isValid = false
                    break
                }
                
                if (i != diagonalIndex) {
                    if (!isRowDot && !colSets[i].add(rowCurrent)) {
                        isValid = false
                        break
                    }
                    
                    if (!isColDot && !rowSets[i].add(colCurrent)) {
                        isValid = false
                        break
                    }
                }
                
                // to find the current diagonal, (col / 3) + 3 * (row / 3)
                val rowBoxIndex = (i/3) + 3 * (row/3)
                val colBoxIndex = (column/3) + 3 * (i/3)
                
                val rowBoxSet = boxSets[rowBoxIndex]
                val colBoxSet = boxSets[colBoxIndex]
                
                if (!isRowDot && !rowBoxSet.add(rowCurrent)) {
                    isValid = false
                    break
                }
                
                if (!isColDot && (column != i) && !colBoxSet.add(colCurrent)) {
                    isValid = false
                    break
                }
            }
            
            diagonalIndex++
        }
        
        return isValid
    }
}

val sol36 = Solution36()

Line_2404.jupyter.kts (5:19 - 20) Variable 'row' initializer is redundant
Line_2404.jupyter.kts (6:22 - 23) Variable 'column' initializer is redundant
Line_2404.jupyter.kts (8:45 - 46) No value passed for parameter 'init'
Line_2404.jupyter.kts (9:45 - 46) No value passed for parameter 'init'
Line_2404.jupyter.kts (10:45 - 46) No value passed for parameter 'init'